<a href="https://colab.research.google.com/github/tejaswishetty17/GenAI_Work/blob/main/Advanced_RAG_Application_Use_case_with_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain langchain-openai langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.3 MB/s eta 0:00

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

#Load the text file, split it into chunks in a logical way. embed it and then store the embeddings in vector db.
load -> split -> embed -> store in vector db.

In [4]:
loader = TextLoader("/content/sample_data/langchain_doc.txt")
raw_docs = loader.load()

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50, separators=["\n\n", "\n", " ", ""])
docs = splitter.split_documents(raw_docs)

In [6]:
for i, chunk in enumerate(docs):
  print(f"\n-- Chunk {i+1}--")
  print(chunk.page_content)


-- Chunk 1--
LangChain Introduction and Core Philosophy:
LangChain offers features and tools that make it easier to work with this concept. In particular, the use of modular abstractions, prebuilt utilities, and community-backed best practices ensures that developers can implement solutions around langchain introduction and core philosophy efficiently. Whether integrating third-party APIs, working with documents, or building agents, LangChain provides end-to-end support for langchain introduction and core philosophy.

-- Chunk 2--
How LangChain Integrates with Large Language Models:
LangChain offers features and tools that make it easier to work with this concept. In particular, the use of modular abstractions, prebuilt utilities, and community-backed best practices ensures that developers can implement solutions around how langchain integrates with large language models efficiently. Whether integrating third-party APIs, working with documents, or building agents, LangChain provides e

In [7]:
embedding_model = OpenAIEmbeddings(model = "text-embedding-3-small")
vectorstore = Chroma.from_documents(docs, embedding_model, persist_directory="./chromadb_1")
retriever = vectorstore.as_retriever()

In [8]:
docs = retriever.invoke("Which vector stores are supported with Langchain?")

In [9]:
def join_chunks(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [10]:
print(join_chunks(docs))

Vector Stores: FAISS, Chroma, Pinecone, and Weaviate:
LangChain offers features and tools that make it easier to work with this concept. In particular, the use of modular abstractions, prebuilt utilities, and community-backed best practices ensures that developers can implement solutions around vector stores: faiss, chroma, pinecone, and weaviate efficiently. Whether integrating third-party APIs, working with documents, or building agents, LangChain provides end-to-end support for vector stores: faiss, chroma, pinecone, and weaviate.

Document Loading and Supported File Formats:
LangChain offers features and tools that make it easier to work with this concept. In particular, the use of modular abstractions, prebuilt utilities, and community-backed best practices ensures that developers can implement solutions around document loading and supported file formats efficiently. Whether integrating third-party APIs, working with documents, or building agents, LangChain provides end-to-end sup

In [11]:
llm = ChatOpenAI(model = "gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant. Answer the question using only the following context:
{context}

Question:{question}
""")

In [12]:
rag_chain = {
    "context":retriever | join_chunks,
    "question": RunnablePassthrough()
} | prompt | llm | StrOutputParser()

In [13]:
query = "What features Langchain offers for prompt engineering?"
response = rag_chain.invoke(query)
print("query:", query)
print("Answer:", response)

query: What features Langchain offers for prompt engineering?
Answer: LangChain offers features and tools that make it easier to work with prompt engineering. In particular, it utilizes modular abstractions, prebuilt utilities, and community-backed best practices to ensure that developers can implement solutions around prompt engineering efficiently. LangChain provides end-to-end support for integrating third-party APIs, working with documents, or building agents in the context of prompt engineering.


pass data as dictionary(to pass multiple context)

In [14]:
from operator import itemgetter
rag_chain = {
    "context":itemgetter("question") | retriever | join_chunks,
    "question": itemgetter("question")
} | prompt | llm | StrOutputParser()

In [15]:
query = "What features Langchain offers for prompt engineering?"
response = rag_chain.invoke({"question":query})
print("query:", query)
print("Answer:", response)

query: What features Langchain offers for prompt engineering?
Answer: LangChain offers features and tools that make it easier to work with prompt engineering. In particular, it utilizes modular abstractions, prebuilt utilities, and community-backed best practices to ensure that developers can implement solutions around prompt engineering efficiently. LangChain provides end-to-end support for integrating third-party APIs, working with documents, or building agents in the context of prompt engineering.
